## Elixir Matrex library for linear regression MNIST digits recognition

Based on Andrew Ng's Coursera course on ML, excercise number 3.

In [17]:
import Matrex

# Define cost function with regularization.
# Take notion of heavy usage of paired functions, which do two matrix operations at one call.
# For example, dot_tn() returns dot product of two matrices, the first of which is being
# transposed. So, you get transposing operation here almost for free.
# And avoid one data copying operation in Elixir.

# Cost function returns cost value (scalar) for the given solution theta
# and gradient values (column matrix).
defmodule LinearRegression do
  def lr_cost_fun(%Matrex{} = theta, {%Matrex{} = x, %Matrex{} = y, lambda} = _params, iteration \\ 0)
      when is_number(lambda) do
    m = y[:rows]

    h = Matrex.dot_and_apply(x, theta, :sigmoid)
    l = Matrex.ones(theta[:rows], theta[:cols]) |> Matrex.set(1, 1, 0)

    regularization =
      Matrex.dot_tn(l, Matrex.square(theta))
      |> Matrex.scalar()
      |> Kernel.*(lambda / (2 * m))

    # Compute the cost and add regularization parameter
    j =
      y
      |> Matrex.dot_tn(Matrex.apply(h, :log), -1)
      |> Matrex.subtract(
        Matrex.dot_tn(
          Matrex.subtract(1, y),
          Matrex.apply(Matrex.subtract(1, h), :log)
        )
      )
      |> Matrex.scalar()
      |> (fn
            :nan -> :nan
            x -> x / m + regularization
          end).()

    # Compute gradient
    grad =
      x
      |> Matrex.dot_tn(Matrex.subtract(h, y))
      |> Matrex.add(Matrex.multiply(theta, l), 1.0, lambda)
      |> Matrex.divide(m)

    {j, grad}
  end
  
  # The same cost function, implemented with  operators from `Matrex.Operators` module.
  # Works 2 times slower, than standard implementation. But it's a way more readable.
  # It is here for demonstrating possibilites of the library.
  def lr_cost_fun_ops(%Matrex{} = theta, {%Matrex{} = x, %Matrex{} = y, lambda} = _params)
      when is_number(lambda) do
    # Turn off original operators. Use this with caution!
    import Kernel, except: [-: 1, +: 2, -: 2, *: 2, /: 2, <|>: 2]
    import Matrex
    import Matrex.Operators
    
    # This line is needed only when used from iex, to remove ambiguity of t/1 function.
    import IEx.Helpers, except: [t: 1]

    m = y[:rows]

    h = sigmoid(x * theta)
    l = ones(size(theta)) |> set(1, 1, 0.0)

    j = (-t(y) * log(h) - t(1 - y) * log(1 - h) + lambda / 2 * t(l) * pow2(theta)) / m

    grad = (t(x) * (h - y) + (theta <|> l) * lambda) / m

    {scalar(j), grad}
  end
end

{:module, LinearRegression, <<70, 79, 82, 49, 0, 0, 16, 0, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 124, 0, 0, 0, 45, 23, 69, 108, 105, 120, 105, 114, 46, 76, 105, 110, 101, 97, 114, 82, 101, 103, 114, 101, 115, 115, 105, 111, ...>>, {:lr_cost_fun_ops, 2}}

In [2]:
# Check working directory to know, where to load data files from.
File.cwd()

{:ok, "/Users/catalyst/Projects/Elixir/IElixir"}

In [3]:

# Load training data (5 000 MNIST digits in 20x20 pixels format)
# A column of ones was added to the beginning of the matrix.
x = Matrex.load("../matrex/test/data/X.mtx.gz")

#Matrex[5000×400]
┌                                                                             ┐
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     ⋮       ⋮       

In [28]:
# Visualize a piece of data
x[1100..1115] |> list_of_rows() |> Enum.map(&(reshape(&1, 20, 20) |> transpose()))  |> reshape(4, 4) |> heatmap()


#Matrex[80×80]
┌                                                                                ┐
│         ▄▄▄                                                                    │
│       ▄▄▄▄▄▄▄▄                ▄ ▄▄▄▄             ▄▄▄▄▄▄        ▄▄▄▄▄▄▄▄▄       │
│       ▄▄▄▄▄▄▄▄▄             ▄▄▄▄▄▄▄▄▄           ▄▄▄▄▄▄▄▄▄       ▄     ▄▄▄▄     │
│        ▄▄▄▄ ▄ ▄▄            ▄▄▄▄ ▄▄▄▄         ▄▄ ▄▄▄▄▄▄                 ▄▄     │
│            ▄▄ ▄▄         ▄▄▄▄▄▄▄▄▄▄▄          ▄▄▄▄ ▄▄▄▄                 ▄▄     │
│    ▄▄▄▄▄▄▄▄▄ ▄▄       ▄▄▄▄▄▄▄▄▄▄▄▄▄▄        ▄▄▄▄▄▄▄▄▄                 ▄▄▄▄     │
│  ▄▄▄▄▄▄▄▄▄▄ ▄▄▄▄     ▄▄▄▄▄▄▄ ▄▄▄▄▄▄▄     ▄▄▄▄▄▄▄▄▄▄▄           ▄▄▄▄▄▄▄▄ ▄▄▄▄▄▄ │
│  ▄ ▄▄▄▄▄▄▄▄▄▄▄▄▄▄    ▄▄▄▄▄▄▄▄▄            ▄▄▄▄▄▄▄▄▄▄          ▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄│
│  ▄▄▄▄▄▄▄ ▄             ▄▄▄▄▄                  ▄  ▄▄▄           ▄▄▄▄▄           │
│                                                                                │
│                            ▄▄                                         

#Matrex[80×80]
┌                                                                             ┐
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …      0.0     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …   8.0e-5     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  …  -0.0007     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00269     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00279     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00278     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00278     0.0     0.0     0.0 │
│     0.0     0.0     0.0     0.0     0.0  … -0.00113     0.0     0.0     0.0 │
│     ⋮       ⋮       ⋮  

In [4]:
# Load labels for each digit. Label 10.0 means zero.
y = Matrex.load("../matrex/test/data/Y.mtx")

#Matrex[5000×1]
┌         ┐
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│    10.0 │
│     ⋮   │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
│     9.0 │
└         ┘

In [5]:
# Initialize theta values. 
theta = Matrex.zeros(x[:cols], 1)

#Matrex[400×1]
┌         ┐
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     ⋮   │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
│     0.0 │
└         ┘

In [6]:
# Regularization parameter and number of learning iterations
lambda = 0.01
iterations = 100

100

In [7]:
# Start learning!
# For speed, learning is done in async streams. 
# It's 'one-vs-all' scheme for each digit.
solutions =
      1..10  # Our ten digits, we wish to recognize
      |> Task.async_stream(
        fn digit ->
          # Prepare labels matrix with only current digit labeled with 1.0
          y3 = Matrex.apply(y, fn val -> if(val == digit, do: 1.0, else: 0.0) end)

          # Use fmincg() optimizer (ported to Elixir with Matrex functions) with previously defined cost function.
          {sX, fX, _i} =
            Matrex.Algorithms.fmincg(&LinearRegression.lr_cost_fun/3, theta, {x, y3, lambda}, iterations)
        
          # Return the digit itself and the best found solution, which is a column matrix 401x1
          {digit, List.last(fX), sX}
        end,
        max_concurrency: 4
      ) # Merge all 10 found solution column matrices into one 10x401 solutions matrix
      |> Enum.map(fn {:ok, {_d, _l, theta}} -> Matrex.to_list(theta) end)
      |> Matrex.new()

#Matrex[10×400]
┌                                                                             ┐
│     0.0     0.0  1.2e-4-0.00114-0.00124  …   -0.076 0.00945     0.0     0.0 │
│     0.0     0.0 -1.1e-4 0.00118 -5.9e-4  …  0.02507 0.00689 -7.9e-4     0.0 │
│     0.0     0.0 -4.0e-5 -7.6e-4 0.01543  …   9.2e-4 -4.0e-5     0.0     0.0 │
│     0.0     0.0 -1.0e-5  7.0e-5 0.00146  … -0.00381  3.2e-4  1.0e-5     0.0 │
│     0.0     0.0     0.0  2.2e-4-0.00259  …  0.01508-0.00138     0.0     0.0 │
│     0.0     0.0     0.0  4.0e-5 -2.4e-4  …  0.00711 -6.6e-4     0.0     0.0 │
│     0.0     0.0 -3.0e-5  2.1e-4  6.1e-4  … -0.00141 -3.5e-4  4.0e-5     0.0 │
│     0.0     0.0 -5.0e-5  4.2e-4  4.9e-4  … -0.00353-0.00278  2.9e-4     0.0 │
│     0.0     0.0     0.0  5.0e-5  2.3e-4  … -0.00216-0.00695  5.8e-4     0.0 │
│     0.0     0.0     0.0 -1.0e-5  1.8e-4  …  0.00171 -3.9e-4  1.0e-5     0.0 │
└                                                                             ┘

In [8]:
# Hope it was fast enough!
# Now let's check the quality of the solution.

# Apply solution to our learning data
predictions =
  x
  |> Matrex.dot_nt(solutions)
  |> Matrex.apply(:sigmoid)

#Matrex[5000×10]
┌                                                                             ┐
│     0.0  1.2e-4  0.0001     0.0  1.2e-4  …      0.0  2.2e-4 0.00116 0.99995 │
│     0.0     0.0  9.0e-5     0.0 0.00612  …      0.0     0.0  1.0e-5 0.99997 │
│     0.0  1.1e-4  4.0e-5     0.0  1.3e-4  …      0.0 0.01634  8.2e-4 0.99792 │
│     0.0  9.0e-5  2.0e-5     0.0     0.0  …      0.0  1.2e-4  1.2e-4 0.99987 │
│     0.0     0.0  1.0e-5     0.0 0.00134  …      0.0  1.0e-5     0.0 0.98695 │
│     0.0  4.0e-5     0.0     0.0  1.0e-5  …      0.0 0.00474     0.0     1.0 │
│     0.0     0.0 0.00978     0.0     0.0  …      0.0  2.0e-5     0.0 0.96467 │
│     0.0 0.33861 0.00333     0.0  0.0693  …      0.0 0.02044     0.0   0.929 │
│     0.0  2.0e-5  1.7e-4     0.0  1.8e-4  …      0.0 0.00713  4.0e-5 0.98945 │
│     0.0     0.0 0.21656     0.0   0.001  …   1.0e-5  4.0e-5 0.00152 0.99808 │
│     0.0     0.0  5.6e-4     0.0 0.15066  …      0.0 0.00306  2.1e-4 0.98681 │
│     ⋮       ⋮       ⋮

In [9]:
# Each row of predictions matrix contains probability that corresponding row of training data X
# contains the image of digit, which is equal to this row's index.
# I.e. algorithm predicts, that first row contains zero,
# beacuse the tenth cell of the row contains the greatest value (0.99952), than other cells.
# And label 10 represents zero in our data.

# Now let's take these predictions, compare them to our labels and count all 
# true predictions (i.e. where prediction for the row is equal to it's label.)
# Then divide this number by the total number of predictions and multiply by 100
# to get the percentage of true predictions, and thus, the accuracy of our learning algorithm.

accuracy =
  1..predictions[:rows]
  |> Enum.reduce(0, fn row, acc ->
    if y[row] == predictions[row][:argmax], do: acc + 1, else: acc
  end)
  |> Kernel./(predictions[:rows])
  |> Kernel.*(100)

95.32000000000001

In [10]:
# Not bad!

nil